# GAP Habitat Models as Species-Level Dictionaries
A script for developing a workflow to get model dictionaries onto ScienceBase

In [2]:
import gapproduction as gp
import pysb

In [3]:
# Where to save json files
outDir = "P:/Proj3/USGap/Vert/Model/2001/GAP_habitat_models/"

In [4]:
# Get list of species to work on
spList = ["bAMREx", "mABBEx", "mammix", "magsqx", "blarbx"]

In [13]:
# Set up your ScienceBase session for operating against. This takes your myUSGS user name and password (could be your usgs.gov credentials or another account depending on what you set up)
sb = pysb.SbSession()
username = input("Username: ")
sb.loginc(str(username))

Username: "nmtarr@ncsu.edu"
········


<pysb.SbSession.SbSession instance at 0x0000000012671A08>

In [15]:
# This is the top level ScienceBase Item ID for the habitat maps
_gapHabitatMapCollectionItem = "527d0a83e4b0850ea0518326"

# This gets all the child IDs from the habitat map collection; returns a list
# you can loop through
habitatMapIDs = sb.get_child_ids(_gapHabitatMapCollectionItem)

# Get list of strUCs from habitatMapIDs to loop on
spUCs = [sb.get_item(x)["identifiers"][0]["key"] for x in habitatMapIDs[:5]]
print(spUCs)

[u'rRNSNx', u'bEASOx', u'bBEWRx', u'mSEWEc', u'rGPSLx']


In [ ]:
# From here you could generate your JSON species model report documents and then upload them right back to the current ScienceBase Item we're dealing with using sb.upload_file_to_item()
counter=0
for species in spUCs:
    counter += 1
    print(species)
    print(str(counter) + " of " + str(len(spUCs)))
    
    # Make an empty dictionary to collect model dictionaries
    speciesDict = {"input_layers":gp.gapmodeling.layers_2001}
    
    # List taxanomic information
    taxonomic = {"common_name":gp.gapdb.NameCommon(species), 
                 "scientific_name":gp.gapdb.NameSci(species),
                 "gap_code": gp.gapdb.Crosswalk(species)[0],
                 "ELCode": gp.gapdb.Crosswalk(species)[1],
                 "ITIS_TSN": gp.gapdb.Crosswalk(species)[2],
                 "Global_SEQ_ID": gp.gapdb.Crosswalk(species)[3]}
    speciesDict["taxonomic"] = taxonomic
    
    # Add placeholder for DOI
    speciesDict["doi"] = "???????"
    
    # Add the species' habitat description
    description = gp.gapmodeling.getHabitatDescription(species)
    speciesDict["habitat_description"] = description
    
    # Add species' references/citations
    referencesDF = gp.gapmodeling.SpReferences(species)
    references = dict(referencesDF["memCitation"])
    
    # Establish a file name for speciesDict
    filename = outDir + species + "_CONUS_HabModel_2001v1.json"
    
    # Get python tuple (like a list) of regional model codes
    modelCodes = [x for x in gp.gapmodeling.ModelCodes(species, 
                                                 publishedOnly=True, 
                                                 conusOnly=True, 
                                                 migratory=False) if int(x[-1:]) in [1,2,3,4,5,6]]
    
    # Get the models as dictionaries, add to a dictionary of models, then add that
    # to the species level dictionary
    models = {}
    for model in modelCodes:
        modelDict = gp.gapmodeling.ModelAsDictionary(model, ecolSystem = "both")
        modelDict["intElevMax"] = 100 # !!!!!!!!!!!!!!!!!  REMOVE
        modelDict["intElevMin"] = 105 # !!!!!!!!!!!!!!!!!  REMOVE
        models[model] = modelDict
    
    speciesDict["models"] = models

print(speciesDict)

rRNSNx
1 of 5
rRNSNx-y6
rRNSNx-y4
rRNSNx-y5
